In [ ]:
import urllib.request
from tqdm import tqdm
from IPython.display import FileLink
import time, os, sys, subprocess

!nvidia-smi
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
!pip install gdown basicsr facexlib gfpgan ffmpeg-python
!pip install -r requirements.txt
!python setup.py develop

import ffmpeg
if not os.path.exists('/kaggle/working/hasil'):
    os.makedirs('/kaggle/working/hasil')
if not os.path.exists('/kaggle/working/done'):
    os.makedirs('/kaggle/working/done')

def download(url, fileinput):
    with urllib.request.urlopen(url) as response, open(fileinput, 'wb') as out_file:
        file_size = int(response.headers.get("Content-Length", 0))
        with tqdm(total=file_size, unit='B', unit_scale=True, unit_divisor=1024, position=0, file=sys.stdout, ncols=80) as pbar:
            while True:
                data = response.read(1024)
                if not data:
                    break
                out_file.write(data)
                pbar.update(len(data))
                
def cekAudio(nama_file):
    video = ffmpeg.probe(nama_file); num_audio_streams = sum(1 for stream in video['streams'] if stream['codec_type'] == 'audio')
    streams = 1 if num_audio_streams == 1 else num_audio_streams if num_audio_streams > 1 else 0
    return streams

reso_flag = "No"; genap_value = None; forceDownload = 0
def cek_reso(panjang, lebar):
    global reso_flag; global genap_value
    for angka in range(150, 301):
        hasil = angka / 100; panjang_hasil = int(panjang * hasil); lebar_hasil = int(lebar * hasil)
        if panjang_hasil % 2 == 0 and lebar_hasil % 2 == 0:
            if lebar_hasil >= 1080:
                reso_flag = "Bisa"; genap_value = hasil; panjang = panjang_hasil; lebar = lebar_hasil
                break
    if lebar_hasil < 1080:
        reso_flag = "No"; genap_value = None
    return reso_flag, genap_value, panjang_hasil, lebar_hasil

In [ ]:
# ================================================= Download Video ======================================================>>
%cd /kaggle/working/
#url = ''
#name, eps, ext = '', '1', '.mkv' 

fileinput = name + '-' + eps + ext
fileoutput = name + '-' + eps + '.mp4'; fileaudio = name + '-' + eps + '.mka'
pathFile = '/kaggle/working/' + fileoutput
#forceDownload = 0

if not os.path.isfile(fileinput):
    download(url, fileinput)
elif forceDownload == 1:
    !rm {fileinput}
    download(url, fileinput)
else:
    print('\033[1m' + 'File Already Exist. Skipping Download....' + '\033[0m')
    
#------------------------------------------------- Checking Parts --------------------------------------------------------||
fileStream = '[2xDA]' if cekAudio(fileinput) > 1 else '[NA]' if cekAudio(fileinput) == 0 else '[2x]'; print('------------------------------------------\n\033[1m' + fileinput + ' => Status : ' + ('Multiple Audio Streams' if cekAudio(fileinput) > 1 else 'Single Audio Stream' if cekAudio(fileinput) == 1 else 'No Audio Stream') + '\033[0m')
inferenceOutput, finalOutput = f"{name}-{eps}_[2x].mp4", f"{name}_{fileStream}_-_{eps}.mkv"; output = subprocess.check_output(f"ffprobe {fileinput} -v error -select_streams v:0 -show_entries stream=width,height -of csv=s=x:p=0", shell=True).decode().strip().split("x"); panjang, lebar = int(output[0]), int(output[1]); reso_flag, scale_value, res_x, res_y = cek_reso(panjang, lebar); 
!ffprobe -v error -select_streams v:0 -count_packets -show_entries stream=nb_read_packets -of csv=p=0 -show_entries format=duration -of default=noprint_wrappers=1 {fileinput}
print("Status:", "\033[1m" + reso_flag + "\033[0m", "Test Scale:", "\033[1m" + str(scale_value) + "\033[0m", "Resolusi X:", panjang, "=>", "\033[1m" + str(res_x) + "\033[0m", "Resolusi Y:", lebar, "=>", "\033[1m" + str(res_y) + "\033[0m"); print(f"\033[1m{'Nilai rekomendasi Scale adalah : ' + str(scale_value) if reso_flag == 'Bisa' else 'Tidak ada nilai yang bisa di scaling'}\033[0m")#; print(inferenceOutput); print(finalOutput)

In [ ]:
!cd /kaggle/working/ && ffprobe -v error -select_streams v:0 -show_entries stream=display_aspect_ratio -of default=noprint_wrappers=1:nokey=1 {fileinput}
# ================================================ Detach Atributes ======================================================>>
print('\033[1m' + 'Converting to .mp4 Without audio....' + '\033[0m'); 
if fileinput.endswith('.mkv') or fileinput.endswith('.avi') or fileinput.endswith('.ogm') or fileinput.endswith('.wmv'):
    !echo y | ffmpeg -i {fileinput} -an -codec copy -vsync 1 {fileoutput}    # for .mkv => Without Audio and no Encode
    #!echo y | ffmpeg -i {fileinput} -c:v libx264 -b:v 1710k -an -vsync 1 {fileoutput}   # -b:v 1883k | for .avi, .ogm, .wmv => Without Audio only + Encode 
elif fileinput.endswith('.mp4'):
    print('\033[1m' + 'Already formatted in .mp4 file. Begin continue to copy audio....' + '\033[0m')
else :
    print('\033[1m' + 'This video format is not supported' + '\033[0m')
#------------------------------------------------- Detaching Audio --------------------------------------------------------||
print('\033[1m' + 'Detaching audio....' + '\033[0m')
!echo y | ffmpeg -i {fileinput} -map 0:a:0 -map 0:a:1? -map 0:a:2? -map 0:a:3? -map 0:s:0? -map 0:s:1? -map 0:s:2? -map 0:s:3? -map 0:s:4? -map 0:s:5? -c copy {fileaudio} && ls -lh 
!echo "[1] Input {fileinput} Video's Duration & Frames:" && ffmpeg -i {fileinput} 2>&1 | grep Duration | sed 's/Duration: \(.*\), start/\1/g' && ffmpeg -i {fileinput} -map 0:v:0 -c copy -f null -y /dev/null 2>&1 | grep -Eo 'frame= *[0-9]+ *' | grep -Eo '[0-9]+' | tail -1 && echo "[2] Converted {fileoutput} Video's Duration & Frames:" && ffmpeg -i {fileoutput} 2>&1 | grep Duration | sed 's/Duration: \(.*\), start/\1/g' && ffmpeg -i {fileoutput} -map 0:v:0 -c copy -f null -y /dev/null 2>&1 | grep -Eo 'frame= *[0-9]+ *' | grep -Eo '[0-9]+' | tail -1

In [ ]:
#================================================= Real-Esrgan Inference =================================================>>
!rm -rf /kaggle/working/hasil/{name}_vidlist.txt /kaggle/working/hasil/{name}_out_tmp_videos /kaggle/working/hasil/{name}_inp_tmp_videos /kaggle/working/hasil/{inferenceOutput}
%cd /kaggle/working/Real-ESRGAN
#scale_value = 1.5
#-------------------- Models -------------------------: realesr-general-x4v3 | realesr-animevideov3 |
!echo y | python inference_realesrgan_video.py -i {pathFile} --outscale {scale_value} -n realesr-animevideov3 --suffix '[2x]' -o /kaggle/working/hasil

In [ ]:
%cd /kaggle/working/hasil/ 
!ls -lh
# ================================================ Converting .mkv =======================================================>>
!echo y | ffmpeg -i {inferenceOutput} -i /kaggle/working/{fileaudio} -map 0:v -map 1:a -map 2:s? -c copy /kaggle/working/{finalOutput}
%cd /kaggle/working/ 
downloadFile = finalOutput.replace('_', ' ')
for root, dirs, files in os.walk('/kaggle/working/'):
    for file in files:
        os.rename(os.path.join(root, file), os.path.join(root, file.replace(finalOutput, downloadFile)))
!ls -lh
print('\033[1m' + 'Click link bellow to Download File ' + '\033[38;5;28m' + downloadFile + '\033[0m')
FileLink(downloadFile)

In [ ]:
%cd /kaggle/working/
storeFiles = 0                          
# ================================================ End Process ===========================================================>>
!rm {fileinput} {fileoutput} {fileaudio} #&& mv "$downloadFile" /kaggle/working/done
print('\033[1m' + 'All temporary video files has been ' + '\033[91m' + 'Deleted !' + '\033[0m')
if storeFiles == 1:
    !mv *.mkv done/
    !ls -lh /kaggle/working/done
    print('\033[1m' + 'All .mkv output files has been ' + '\033[38;5;129m' + 'Stored !' + '\033[0m')

In [ ]:
#Splitting
!echo y | ffmpeg -i .mp4 -ss 0 -to 60 -c:v copy .mp4